# IMDB DATASET ASSIGNMENT

In [1]:
import sqlite3

In [2]:
db = 'Db-IMDB.db'
conn = sqlite3.connect(db)

In [3]:
c = conn.cursor()

#### Q1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [4]:
query1 = '''SELECT DISTINCT TRIM(p.Name) AS 'Director_Name', m.title AS 'Movie_Title', m.year AS 'Year'
FROM Movie m 
LEFT JOIN M_Director md
ON m.MID = md.MID
LEFT JOIN Person p
ON md.PID = p.PID
WHERE (m.MID IN (SELECT mg.MID FROM M_Genre mg WHERE mg.GID IN
                    (SELECT g.GID FROM Genre g WHERE g.Name LIKE '%Comedy%')
                )
        ) 
        AND 
        ((m.year % 4) == 0) LIMIT 10'''

In [5]:
print('(\'Director Name\'), (\'Movie Title\'), (\'Year\')')
for row in c.execute(query1):
    print(row)

('Director Name'), ('Movie Title'), ('Year')
('Milap Zaveri', 'Mastizaade', '2016')
('Danny Leiner', 'Harold & Kumar Go to White Castle', '2004')
('Anurag Kashyap', 'Gangs of Wasseypur', '2012')
('Frank Coraci', 'Around the World in 80 Days', '2004')
('Griffin Dunne', 'The Accidental Husband', '2008')
('Anurag Basu', 'Barfi!', '2012')
('Gurinder Chadha', 'Bride & Prejudice', '2004')
('Mike Judge', 'Beavis and Butt-Head Do America', '1996')
('Abhinay Deo', 'Blackmail', 'I 2018')
('Tarun Mansukhani', 'Dostana', '2008')


#### Q2. List the names of all the actors who played in the movie 'Anand' (1971)

In [6]:
query2 = '''SELECT p.Name AS 'Actor Name in Movie Anand'
FROM Person p WHERE p.PID IN
    (SELECT TRIM(mc.PID)
    FROM M_Cast mc JOIN Movie m
    ON mc.MID = m.MID
    WHERE m.title = 'Anand')
'''

In [7]:
print('Actor Name in Movie Anand')
for row in c.execute(query2):
    print(row)

Actor Name in Movie Anand
(' Amitabh Bachchan',)
(' Rajesh Khanna',)
(' Sumita Sanyal',)
(' Ramesh Deo',)
(' Seema Deo',)
(' Asit Kumar Sen',)
(' Dev Kishan',)
(' Atam Prakash',)
(' Lalita Kumari',)
(' Savita',)
(' Brahm Bhardwaj',)
(' Gurnam Singh',)
(' Lalita Pawar',)
(' Durga Khote',)
(' Dara Singh',)
(' Johnny Walker',)
(' Moolchand',)


#### Q3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [8]:
query3 = '''SELECT p.Name
FROM Person p 
JOIN M_Cast mc on p.PID = TRIM(mc.PID) 
JOIN Movie m ON mc.MID = m.MID
WHERE m.year < 1970
INTERSECT
SELECT p.Name
FROM Person p 
JOIN M_Cast mc on p.PID = TRIM(mc.PID) 
JOIN Movie m ON mc.MID = m.MID
WHERE m.year > 1990
LIMIT 10'''

In [9]:
print('Actor Name between 1970 AND 1990')
for row in c.execute(query3):
    print(row)

Actor Name between 1970 AND 1990
(' A.K. Hangal',)
(' Aachi Manorama',)
(' Abbas',)
(' Abdul',)
(' Abhi Bhattacharya',)
(' Abhimanyu Sharma',)
(' Achala Sachdev',)
(' Adil',)
(' Ajay',)
(' Ajit',)


#### Q4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [10]:
query4 = '''SELECT p.Name AS 'Movie Director', COUNT(md.PID) movie_count
            FROM Person p 
            JOIN M_Director md
            ON p.PID = md.PID
            GROUP BY md.PID
            HAVING movie_count > 9
            ORDER BY movie_count
            DESC LIMIT 10'''

In [11]:
print('Movie Director, Number of Movies Directed')
for row in c.execute(query4):
    print(row)

Movie Director, Number of Movies Directed
('David Dhawan', 78)
('Mahesh Bhatt', 70)
('Ram Gopal Varma', 60)
('Vikram Bhatt', 58)
('Hrishikesh Mukherjee', 54)
('Yash Chopra', 42)
('Basu Chatterjee', 38)
('Shakti Samanta', 38)
('Subhash Ghai', 36)
('Shyam Benegal', 34)


#### Q5 A. For each year, count the number of movies in that year that had only female actors.

In [12]:
query_5_a = '''CREATE VIEW temp_5_a AS
SELECT m.title, SUBSTR(m.year, LENGTH(m.year)-3, 4) AS 'Year', p.Name, p.Gender
FROM Movie m 
JOIN M_Cast mc ON m.MID = mc.MID
JOIN Person p ON TRIM(mc.PID) = p.PID'''

In [13]:
query_5_a_a = '''CREATE VIEW temp_5_a_a AS
SELECT DISTINCT title, Year 
FROM temp_5_a
WHERE title NOT IN (SELECT title 
FROM temp_5_a
WHERE Gender = 'Male')'''

In [14]:
query_5a = '''SELECT Year, COUNT(Year) FROM temp_5_a_a
GROUP BY Year
ORDER BY COUNT(Year) DESC'''

In [16]:
print('Year, Movie Count')
for row in c.execute(query_5a):
    print(row)

Year, Movie Count
('2018', 2)
('1939', 1)
('1999', 1)
('2000', 1)


#### Q5 B. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [17]:
query_5_b = '''CREATE VIEW temp_5_b_movies_with_only_female AS
SELECT Year, COUNT(Year) AS 'movie_count' FROM temp_5_a_a
GROUP BY Year
ORDER BY COUNT(Year) DESC'''

In [18]:
query_3_b_b = '''CREATE VIEW temp_5_b_movies_in_year AS
SELECT DISTINCT title, Year 
FROM temp_5_a'''

In [19]:
query_3_b_b_b = '''CREATE VIEW temp_5_b_total_movies_in_year AS
SELECT Year, COUNT(Year) AS 'movie_count' FROM temp_5_b_movies_in_year
GROUP BY Year
ORDER BY COUNT(Year) DESC'''

In [20]:
query_5b = '''SELECT a.Year, ((a.movie_count*100.0)/b.movie_count) AS 'Female Percenatge', b.movie_count AS 'Total Movie Count'
FROM temp_5_b_movies_with_only_female a JOIN temp_5_b_total_movies_in_year b ON a.Year = b.Year
ORDER BY a.YEAR DESC'''

In [21]:
print('Year, Percentage Female, Total Movie Count')
for row in c.execute(query_5b):
    print(row)

Year, Percentage Female, Total Movie Count
('2018', 1.9230769230769231, 104)
('2000', 1.5625, 64)
('1999', 1.5151515151515151, 66)
('1939', 50.0, 2)


#### Q6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [22]:
query6 = '''SELECT m.title AS 'Movie Name', (SELECT DISTINCT COUNT(MID) FROM M_Cast GROUP BY MID ORDER BY COUNT(PID) DESC) AS 'Cast No.'
FROM Movie  m
WHERE m.MID 
IN (SELECT DISTINCT MID 
FROM M_Cast 
GROUP BY MID 
ORDER BY COUNT(PID) DESC 
LIMIT 1)'''

In [23]:
print('Movie Name, Cast Count')
for row in c.execute(query6):
    print(row)

Movie Name, Cast Count
("Ocean's Eight", 238)


#### Q7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [24]:
query7 = '''SELECT m1.year AS 'FROM', m1.year+9 AS 'TO', COUNT(*) AS 'Movie in Decade'
FROM (SELECT DISTINCT year FROM Movie) m1 JOIN Movie m2 ON m1.year > m2.year AND m1.year < m2.year+10
GROUP BY m1.year
ORDER BY COUNT(*) DESC
LIMIT 1'''

In [25]:
print('FROM, TO, Max Movies')
for row in c.execute(query7):
    print(row)

FROM, TO, Max Movies
('2018', 2027, 1025)


#### Q8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [26]:
query8 = '''SELECT PID, Name 
FROM Person 
WHERE  PID NOT IN (SELECT DISTINCT PID 
FROM M_Cast AS C1 NATURAL JOIN Movie as M1 
WHERE EXISTS (SELECT MID 
FROM M_Cast AS C2 NATURAL JOIN Movie AS M2 
WHERE C1.PID = C2.PID AND (M2.year-3) > M1.year AND NOT EXISTS (SELECT MID 
FROM M_Cast AS C3 NATURAL JOIN Movie AS M3 
WHERE C1.PID = C3.PID AND M1.year < M3.year AND M3.year < M2.year)))
LIMIT 10'''

In [27]:
for row in c.execute(query8):
    print(row)

('nm0000288', ' Christian Bale')
('nm0000949', ' Cate Blanchett')
('nm1212722', ' Benedict Cumberbatch')
('nm0365140', ' Naomie Harris')
('nm0785227', ' Andy Serkis')
('nm0611932', ' Peter Mullan')
('nm2930503', ' Jack Reynor')
('nm0550371', ' Eddie Marsan')
('nm0390903', ' Tom Hollander')
('nm0722629', ' Matthew Rhys')


#### Q9. Find all the actors that made more movies with Yash Chopra than any other director.

In [28]:
query9 = '''SELECT p1.PID, TRIM(p1.Name), COUNT(m1.MID) AS movies_with_yc
FROM Person p1 JOIN M_Cast mc1 ON p1.PID = TRIM(mc1.PID) JOIN Movie m1 ON mc1.MID = m1.MID JOIN M_Director md1 ON m1.MID = md1.MID JOIN Person p2 ON md1.PID = p2.PID
WHERE p2.Name = 'Yash Chopra'
GROUP BY p1.PID
HAVING COUNT(m1.MID) > (SELECT COUNT(m4.MID) FROM Person p3 JOIN M_Cast mc2 ON p3.PID = TRIM(mc2.PID) JOIN Movie m4 ON mc2.MID = m4.MID JOIN M_Director md2 ON m4.MID = md2.MID JOIN Person AS p4 ON md2.PID = p4.PID
WHERE p1.PID = p3.PID AND p4.Name != 'Yash Chopra'
GROUP BY p4.PID)
ORDER BY movies_with_yc DESC
LIMIT 10'''

In [29]:
print('PID, Name of Actors, Number of Movies')
for row in c.execute(query9):
    print(row)

PID, Name of Actors, Number of Movies
('nm0471443', 'Manmohan Krishna', 20)
('nm0004434', 'Shashi Kapoor', 14)
('nm0451600', 'Anupam Kher', 14)
('nm0707271', 'Jagdish Raj', 11)
('nm0407002', 'Iftekhar', 9)
('nm0004564', 'Hema Malini', 8)
('nm0025630', 'Vikas Anand', 8)
('nm0534501', 'Madan Puri', 8)
('nm0894340', 'Deven Verma', 8)
('nm0000821', 'Amitabh Bachchan', 6)


#### Q10. The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [30]:
query10 = '''SELECT DISTINCT p1.PID, TRIM(p1.Name) AS 'Shahrukh No 2'
FROM Person p1 JOIN M_Cast mc1 ON p1.PID = TRIM(mc1.PID)
WHERE TRIM(p1.Name) <> 'Shah Rukh Khan' AND mc1.MID IN (SELECT mc2.MID 
FROM M_Cast mc2
WHERE TRIM(mc2.PID) IN (SELECT p2.PID 
FROM Person p2 JOIN M_Cast mc3 ON p2.PID = TRIM(mc3.PID)
WHERE TRIM(p2.Name) <> 'Shah Rukh Khan' AND mc3.MID IN (SELECT mc4.MID 
FROM Person p3 JOIN M_Cast mc4 
WHERE TRIM(p3.Name) = 'Shah Rukh Khan'))) 
AND p1.PID NOT IN (SELECT p4.PID 
FROM Person p4 JOIN M_Cast mc5 ON p4.PID = TRIM(mc5.PID)
WHERE TRIM(p4.Name) <> 'Shah Rukh Khan'
AND mc5.MID IN (SELECT mc6.MID 
FROM Person p5 JOIN M_Cast mc6 ON p5.PID = TRIM(mc6.PID)
WHERE TRIM(p5.Name) = 'Shah Rukh Khan'))
LIMIT 10'''

In [31]:
print('PID, Shahrukh No. 2')
for row in c.execute(query10):
    print(row)

PID, Shahrukh No. 2
('nm0000288', 'Christian Bale')
('nm0000949', 'Cate Blanchett')
('nm1212722', 'Benedict Cumberbatch')
('nm0365140', 'Naomie Harris')
('nm0785227', 'Andy Serkis')
('nm0611932', 'Peter Mullan')
('nm2930503', 'Jack Reynor')
('nm0550371', 'Eddie Marsan')
('nm0390903', 'Tom Hollander')
('nm0722629', 'Matthew Rhys')


In [32]:
conn.close()